In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [2]:
df=pd.read_csv("/kaggle/input/novozymes-enzyme-stability-prediction/train.csv")
df=df.drop(columns=["data_source"])

In [3]:
df["protein_sequence"].nunique()

28981

In [4]:
df.isnull().sum()

seq_id                0
protein_sequence      0
pH                  286
tm                    0
dtype: int64

In [5]:
label_encoder=LabelEncoder()
df["protein_sequence"]=label_encoder.fit_transform(df["protein_sequence"])

In [6]:
df["pH"]=df["pH"].fillna(0)

In [7]:
one_hot=OneHotEncoder(sparse_output=False)
one=one_hot.fit_transform(df[["protein_sequence"]])
one=pd.DataFrame(one,columns=one_hot.get_feature_names_out())
one["seq_id"]=df["seq_id"]
one["pH"]=df["pH"]

In [8]:
one.shape

(31390, 28983)

In [9]:
df

,seq_id,protein_sequence,pH,tm
0,0,0,7.0,75.7
1,1,1,7.0,50.5
2,2,2,7.0,40.5
3,3,3,7.0,47.2
4,4,4,7.0,49.5
...,...,...,...,...
31385,31385,28976,7.0,51.8
31386,31386,28977,7.0,37.2
31387,31387,28978,7.0,64.6
31388,31388,28979,7.0,50.7


In [10]:
df.isnull().sum()

seq_id              0
protein_sequence    0
pH                  0
tm                  0
dtype: int64

In [11]:
X=df.drop(columns=["tm"])
Y=df["tm"]

In [12]:
X_train,X_test,Y_train,Y_test=train_test_split(one,Y,test_size=0.2,random_state=42)

In [13]:
# random_reg=RandomForestRegressor(n_estimators=50)
# random_reg.fit(X_train,Y_train)
# random_reg.score(X_test,Y_test)

# # Deep learning

In [21]:
from keras.layers import Dense,Input,BatchNormalization,Dropout
from keras.models import Model
from keras.initializers import HeNormal
from keras.regularizers import l2,l1_l2
from keras.callbacks import EarlyStopping

In [15]:
early_stopping=EarlyStopping(
monitor='val_loss',
patience=5,
verbose=1)

In [16]:
X_train.shape

(25112, 28983)

In [31]:
inputs=Input(shape=(28983,))
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(inputs)
D=Dropout(0.5)(D)
D=Dense(32,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)

B=BatchNormalization()(D)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
D=Dropout(0.5)(D)
D=Dense(64,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)

# B=BatchNormalization()(D)
# D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
# D=Dropout(0.5)(D)
# D=Dense(128,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)

# B=BatchNormalization()(D)
# D=Dense(56,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(B)
# D=Dropout(0.5)(D)
# D=Dense(56,activation="relu",kernel_initializer=HeNormal(),kernel_regularizer=l2(0.01))(D)
outputs=Dense(1)(D)
model=Model(inputs,outputs)
model.compile(optimizer='adam',loss='mse',metrics=["mae"])

In [32]:
model.fit(X_train,Y_train,epochs=50,batch_size=32,validation_data=(X_test,Y_test),callbacks=[early_stopping])

Epoch 1/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 16s 13ms/step - loss: 891.1030 - mae: 23.3217 - val_loss: 220.9026 - val_mae: 10.5381
Epoch 2/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 275.7630 - mae: 12.3888 - val_loss: 200.7392 - val_mae: 9.7591
Epoch 3/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 256.8743 - mae: 11.8277 - val_loss: 211.0526 - val_mae: 10.0890
Epoch 4/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 242.6180 - mae: 11.3501 - val_loss: 226.5453 - val_mae: 10.6104
Epoch 5/50
785/785 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 230.1743 - mae: 10.9232 - val_loss: 232.5781 - val_mae: 10.8612
Epoch 5: early stopping


In [33]:
loss,mse=model.evaluate(X_test,Y_test)

197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 241.4536 - mae: 11.0460
